In [1]:
import os
import re
import yaml

import pandas as pd

In [2]:
root_dir = '../../data'
with open('catalogue.yaml') as f:
    files = yaml.safe_load(f)

In [85]:
def make_key(s: str, sep='-'):
      return re.sub(r'\W+', sep, s.lower()).strip(sep)

class Dimension:
    def __init__(self, series: pd.Series):
        self.name = series.name
        self.values = series.unique().tolist()

    def __repr__(self):
        return f'Dimension->{self.name}'
    
    def to_dict(self):
        return self.__dict__

class Fact:
    def __init__(self, series: pd.Series):
        self.name = series.name
        self.type = str(series.dtype)
        self.description = series.describe().to_dict()
        self.na = series.isna().sum()

    def __repr__(self):
        return f'Fact->{self.name} ({self.type})'

    def to_dict(self):
        return self.__dict__

class Metadata:
    def load(self, dataset_path: pd.DataFrame, loader=pd.read_csv, variables=None, values=None, ignored=None, root_dir='', id=None, group=None):
        if variables is None:
              variables = ['variable_name']
        if values is None:
              values = ['value']
        if ignored is None:
              ignored = []

        self.id = id or make_key(os.path.basename(dataset_path))
        self.group = group or os.path.dirname(dataset_path).replace(os.sep, '.')

        dataset: pd.DataFrame = loader(os.path.join(root_dir, dataset_path))

        # Process variables
        fact_columns = variables + values
        dimension_columns = [c for c in dataset.columns.to_list() if c not in fact_columns + ignored]

        # Clean up duplicates
        dataset = dataset[~dataset.duplicated(subset=dimension_columns)]

        try:
              facts = dataset.pivot(index=dimension_columns, columns=variables, values=values).reset_index(drop=True)
        except Exception:
              return None
        facts.columns = facts.columns.droplevel()
        self.facts = [Fact(f) for _, f in facts.items()]

        # Calculate dimension columns
        self.dimensions = [Dimension(x[1]) for x in dataset.loc[:, dimension_columns].items()]

        return self
  
    def __repr__(self):
        return f'Metadata->{self.dimensions}->{self.facts}'

    def to_dict(self):
        result = {k: v for k, v in self.__dict__.items() if k not in ['facts', 'dimensions']}
        result['facts'] = [f.to_dict() for f in self.facts]
        result['dimensions'] = [d.to_dict() for d in self.dimensions]
        return result


catalogue = [
  Metadata().load(f['file'], root_dir=root_dir, id=f['view'], group=f['dataset']) for f in files
]

In [86]:

os.makedirs('../../src/_data/metadata/', exist_ok=True)

output = pd.DataFrame([
        entry.to_dict() for entry in catalogue if entry
]).merge(
    pd.DataFrame(files), left_on=['group', 'id'], right_on=['dataset', 'view'],
    how='right'
).set_index(
    ['dataset', 'view']
).drop(
    columns=['group', 'id']
).sort_values(
    by=['dataset', 'view']
).reset_index(
)

output.to_json(
  '../../src/_data/metadata/catalogue.json',
  orient='records',
  indent=2,
)

In [87]:
output

,dataset,view,facts,dimensions,file,name,type,pipeline,source,description
0,HB,claimants,"[{'name': 'HB_claimants', 'type': 'float64', '...","[{'name': 'Unnamed: 0', 'values': [0, 1, 2, 3,...",HB/claimants.csv,Housing benefit claimants,.csv,NaN,NaN,NaN
1,ashe,weekly-earnings,"[{'name': 'median_weekly_wage', 'type': 'float...","[{'name': 'date', 'values': [2022]}, {'name': ...",ashe/weekly-earnings.csv,Annual Survey of Hours and Earnings Weekly Ear...,.csv,NaN,NaN,NaN
2,claimant-count,claimant-count,"[{'name': 'Claimant count', 'type': 'float64',...","[{'name': 'date', 'values': ['2023-08']}, {'na...",claimant-count/claimant-count.csv,Universal Credit claimant count,.csv,NaN,NaN,NaN
3,clif,clif_ABS,"[{'name': 'children_in_low_income', 'type': 'f...","[{'name': 'Year', 'values': ['2021/22']}, {'na...",clif/clif_ABS.csv,Children in low-income families ABS,.csv,NaN,NaN,NaN
4,clif,clif_REL,"[{'name': 'children_in_low_income', 'type': 'f...","[{'name': 'Year', 'values': ['2021/22']}, {'na...",clif/clif_REL.csv,Children in low-income families REL,.csv,NaN,NaN,NaN
5,council-tax-support,council-tax-support,"[{'name': 'pensioners', 'type': 'float64', 'de...","[{'name': 'date', 'values': ['2015-06-30', '20...",council-tax-support/council-tax-support.csv,NaN,.csv,NaN,NaN,NaN
6,free-school-meals,fsm_time_series,NaN,NaN,free-school-meals/fsm_time_series.csv,Free School Meals,.csv,NaN,NaN,NaN
7,fuel-poverty,fuel-poverty,"[{'name': 'Number of households', 'type': 'flo...","[{'name': 'date', 'values': [2022]}, {'name': ...",fuel-poverty/fuel-poverty.csv,Fuel Poverty,.csv,NaN,NaN,NaN
8,gva,gva,"[{'name': 'GVA', 'type': 'float64', 'descripti...","[{'name': 'date', 'values': ['1998-01-01', '19...",gva/gva.csv,Gross Value Added,.csv,NaN,NaN,Gross value added (GVA) is an economic product...
9,hbai,by_age_category,"[{'name': 'population', 'type': 'float64', 'de...","[{'name': 'Financial Year', 'values': ['1994-9...",hbai/by_age_category.csv,HBAI by age category,.csv,pipelines/statxplore/hbai.ipynb,NaN,NaN
